In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#importing the data set:
dataframes = pd.read_csv("forestfires.csv")

#data proprocessing:

#replacing string values by numerical values:
dataframes.month.replace(('jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'),
                        (1,2,3,4,5,6,7,8,9,10,11,12), inplace=True)
dataframes.day.replace(('mon','tue','wed','thu','fri','sat','sun'),
                       (1,2,3,4,5,6,7), inplace=True)
from sklearn.model_selection import train_test_split
# split original data into training and test set 80/20
train, test = train_test_split(dataframes, test_size=0.2)

# separate X and target for both train and test
train_target = train.pop('area')
test_target = test.pop('area')
#convert from pd to np for easier modification
train_target = np.array(train_target)
test_target = np.array(test_target)
train = np.array(train)
test = np.array(test)
#find all the nonzero indexes
non_zero_areas = train_target.nonzero()
#copy by values, without these every thing will be pass by reference
#classification variables
train_classification_target = 1*train_target
train_classification = 1*train
test_classification_target = 1*test_target
test_classification = 1*test
#regression variables
train_regression_target = 1*train_target
train_regression = 1*train
test_regression_target = 1*test_target
test_regression = 1*test
#set all nonzero to 1
for i in non_zero_areas:
    train_classification_target[i] = 1;
for i in test_target.nonzero():
    test_classification_target[i] = 1   
#perform preprocessing. refer to the main regression code for detail.
from sklearn.preprocessing import StandardScaler, Normalizer
ssc = StandardScaler()
train_classification = ssc.fit_transform(train_classification)
test_classification = ssc.transform(test_classification)

nmc = Normalizer()
train_classification = nmc.fit_transform(train_classification)
test_classification = nmc.transform(test_classification)

In [0]:
#Decision Tree
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_classification, train_classification_target)

In [0]:
#SVM
from sklearn import svm
clfsvm = svm.SVC()
clfsvm.fit(train_classification, train_classification_target)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
#NN
from sklearn.neural_network import MLPClassifier
clfnn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clfnn.fit(train_classification, train_classification_target)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
#Random Forest
from sklearn import ensemble
clfrf = ensemble.RandomForestClassifier()
clfrf.fit(train_classification, train_classification_target)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
#accuracy of each model
from sklearn.metrics import accuracy_score
models = []
models.append(('Decision Tree',clf))
models.append(('SVM',clfsvm))
models.append(('NN',clfnn))
models.append(('Random Forest',clfrf))

for name, mol in models:
    print(name)
    print(accuracy_score(mol.predict(test_classification), test_classification_target))

Decision Tree
0.5096153846153846
SVM
0.5288461538461539
NN
0.5769230769230769
Random Forest
0.5576923076923077


In [0]:
#Tin Nguyen
#remove all the zero values
for i in np.where(train_regression_target==0):
    train_regression = np.delete(train_regression, i, axis=0)
    train_regression_target = np.delete(train_regression_target, i)
for i in np.where(test_regression_target==0):
    test_regression = np.delete(test_regression, i, axis=0)
    test_regression_target = np.delete(test_regression_target, i)
    
#preprocessing
ss = StandardScaler()
train_regression = ss.fit_transform(train_regression)
test_regression = ss.transform(test_regression)

nm = Normalizer()
train_regression = nm.fit_transform(train_regression)
test_regression = nm.transform(test_regression)
#fitting
from sklearn.svm import SVR
rgn = SVR()            
rgn.fit(train_regression, train_regression_target)

from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(rgn.predict(test_regression), test_regression_target))

10.950389680441672


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
